In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Asus\\vs_code\\brain_tumor_Unet\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\vs_code\\brain_tumor_Unet'

In [18]:
from src.brain_tumor.constants import *
from src.brain_tumor.utils.common import read_yaml, create_directories
from pathlib import Path

In [ ]:
model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from src.brain_tumor.entity.config_entity import PrepareCallbacksConfig

In [22]:
class ConfugarationManager:

    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        
        # Assuming read_yaml and create_directories are custom functions in your module
        self.config_ = read_yaml(config_file_path)
        self.params_ = read_yaml(params_file_path)
        create_directories([self.config_.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        confg_          = self.config_.prepare_callbacks
        model_ckpt_dir  = confg_.checkpoint_model_filepath
        create_directories([
            Path(os.path.dirname(model_ckpt_dir)),
            Path(confg_.tensorboard_root_log_dir)
            ])
        prepare_callbacks_config        = PrepareCallbacksConfig(
            root_dir                    = Path(confg_.root_dir),
            tensorboard_root_log_dir    = Path(confg_.tensorboard_root_log_dir),
            checkpoint_model_filepath   = Path(confg_.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [21]:
config = ConfugarationManager()
path = config.get_prepare_callback_config()

In [23]:


import os
import tensorflow as tf
import time



In [29]:
time_stamp = time.strftime("%m-%d-%Y-%H-%M-%S")

In [44]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        time_stamp          = time.strftime("%m-%d-%Y-%H-%M-%S")
        tb_running_log_dir  = self.config.tensorboard_root_log_dir
        tb_running_log_dir  = os.path.join(tb_running_log_dir,
                                           f"tb_logs_at_{time_stamp}")

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,
                                                  save_best_only=True)

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [45]:
config = ConfugarationManager()
prepare_callbacks_config = config.get_prepare_callback_config()
prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)

In [47]:
prepare_callbacks.get_tb_ckpt_callbacks()